In [1]:
import os
import pandas as pd
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import torchvision.transforms as transforms
import re

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [ ]:
class Vocabulary:
    def __init__(self, freq_threshold):
        self.itos = {0: "<PAD>", 1: "<SOS>", 2: "<EOS>", 3: "<UNK>"}
        self.stoi = {"<PAD>": 0, "<SOS>": 1, "<EOS>": 2, "<UNK>": 3}
        self.freq_threshold = freq_threshold

    def __len__(self):
        return len(self.itos)

    @staticmethod
    def tokenizer_eng(text):
        text = text.lower()
        text = re.sub(r'([.,!?"\'-])', r' \1 ', text)
        return text.split()

    def build_vocabulary(self, sentence_list):
        frequencies = {}
        idx = 4 # Start index for new words (0-3 are taken by special tokens)

        for sentence in sentence_list:
            for word in self.tokenizer_eng(sentence):
                if word not in frequencies:
                    frequencies[word] = 1
                else:
                    frequencies[word] += 1

                if frequencies[word] == self.freq_threshold:
                    self.stoi[word] = idx
                    self.itos[idx] = word
                    idx += 1

    def numericalize(self, text):
        tokenized_text = self.tokenizer_eng(text)

        return [
            self.stoi[token] if token in self.stoi else self.stoi["<UNK>"]
            for token in tokenized_text
        ]

In [ ]:
class FlickrDataset(Dataset):
    def __init__(self, root_dir, captions_file, transform=None, freq_threshold=5):
        self.root_dir = root_dir
        self.df = pd.read_csv(captions_file)
        self.transform = transform

        self.imgs = self.df["image"]
        self.captions = self.df["caption"]

        self.vocab = Vocabulary(freq_threshold)
        self.vocab.build_vocabulary(self.captions.tolist())

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        caption = self.captions[index]
        img_id = self.imgs[index]
        
        img = Image.open(os.path.join(self.root_dir, "images", img_id)).convert("RGB")

        if self.transform is not None:
            img = self.transform(img)

        numericalized_caption = [self.vocab.stoi["<SOS>"]]
        numericalized_caption += self.vocab.numericalize(caption)
        numericalized_caption.append(self.vocab.stoi["<EOS>"])

        return img, torch.tensor(numericalized_caption)

In [ ]:
class MyCollate:
    def __init__(self, pad_idx):
        self.pad_idx = pad_idx

    def __call__(self, batch):
        imgs = [item[0] for item in batch]
        captions = [item[1] for item in batch]

        # (batch_size, 3, 224, 224)
        imgs = torch.stack(imgs, dim=0)

        targets = pad_sequence(captions, batch_first=True, padding_value=self.pad_idx)

        return imgs, targets

In [9]:
# 1. Define Transforms (Resize to 224x224 for ResNet, Convert to Tensor, Normalize)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 2. Setup Data Loader
dataset = FlickrDataset(
    root_dir="caption_data", 
    captions_file="caption_data/captions.txt", 
    transform=transform
)

# Padding index is usually 0
pad_idx = dataset.vocab.stoi["<PAD>"]

# 3. Create the Loader
loader = DataLoader(
    dataset=dataset,
    batch_size=32,
    num_workers=0,
    shuffle=True,
    collate_fn=MyCollate(pad_idx=pad_idx)
)

# 4. Print Stats to share with your team
print(f"Vocabulary Size: {len(dataset.vocab)}")

# Grab one batch to check shapes
for images, captions in loader:
    print(f"Batch Image Shape: {images.shape}") # Expect: [32, 3, 224, 224]
    print(f"Batch Caption Shape: {captions.shape}") # Expect: [32, Max_Len]
    print("Example Caption (Numerical):", captions[0])
    break

Vocabulary Size: 2994
Batch Image Shape: torch.Size([32, 3, 224, 224])
Batch Caption Shape: torch.Size([32, 20])
Example Caption (Numerical): tensor([  1,   4, 465,  17, 108,  40,   4,  31, 861,   5,   2,   0,   0,   0,
          0,   0,   0,   0,   0,   0])
